In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
from pixelmed_calc.medical_imaging.modelstore import get_model_from_name
from pixelmed_calc.medical_imaging.DLtools.process import load_data_from_csv,get_file_and_labels,get_transforms,train_and_validate
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
from monai.data import ImageDataset
import os
import pandas as pd
from torch.utils.data import DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F


# CSV文件路径
csv_path = r'E:\127lungnode\lungnode\group.csv'
train_base_path = r'E:\127lungnode\lungnode\crop3d'
val_base_path = r'E:\127lungnode\lungnode\crop3d'

tfs=get_transforms()
# 读取所有数据
file_names, labels, groups = load_data_from_csv(csv_path)
# 获取训练数据和标签（可以设置 base_path 为 None 或实际路径）
file_train, train_labels = get_file_and_labels(file_names, labels, groups, 'train', train_base_path)
# 获取验证数据和标签（可以设置 base_path 为 None 或实际路径）
file_val, val_labels = get_file_and_labels(file_names, labels, groups, 'val', val_base_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = get_model_from_name['resnet50_3D'](num_classes=2, pretrained=False, in_channels=1).to(device)
model.load_state_dict(torch.load(r'd:\pixelmedAI\function_pm\note2-深度学习分类\results\latest_model.pth'))

In [2]:
for name, _ in model.named_modules(): print(name)


conv1
bn1
relu
maxpool
layer1
layer1.0
layer1.0.conv1
layer1.0.bn1
layer1.0.conv2
layer1.0.bn2
layer1.0.conv3
layer1.0.bn3
layer1.0.relu
layer1.0.downsample
layer1.0.downsample.0
layer1.0.downsample.1
layer1.1
layer1.1.conv1
layer1.1.bn1
layer1.1.conv2
layer1.1.bn2
layer1.1.conv3
layer1.1.bn3
layer1.1.relu
layer1.2
layer1.2.conv1
layer1.2.bn1
layer1.2.conv2
layer1.2.bn2
layer1.2.conv3
layer1.2.bn3
layer1.2.relu
layer2
layer2.0
layer2.0.conv1
layer2.0.bn1
layer2.0.conv2
layer2.0.bn2
layer2.0.conv3
layer2.0.bn3
layer2.0.relu
layer2.0.downsample
layer2.0.downsample.0
layer2.0.downsample.1
layer2.1
layer2.1.conv1
layer2.1.bn1
layer2.1.conv2
layer2.1.bn2
layer2.1.conv3
layer2.1.bn3
layer2.1.relu
layer2.2
layer2.2.conv1
layer2.2.bn1
layer2.2.conv2
layer2.2.bn2
layer2.2.conv3
layer2.2.bn3
layer2.2.relu
layer2.3
layer2.3.conv1
layer2.3.bn1
layer2.3.conv2
layer2.3.bn2
layer2.3.conv3
layer2.3.bn3
layer2.3.relu
layer3
layer3.0
layer3.0.conv1
layer3.0.bn1
layer3.0.conv2
layer3.0.bn2
layer3.0.conv

In [3]:
import numpy as np
import pandas as pd
from pixelmed_calc.medical_imaging.DLtools.utils import extract_layer_output
val_ds_viz = ImageDataset(image_files=file_val, transform=tfs)  
val_loader_viz = DataLoader(val_ds_viz, batch_size=1, shuffle=False, num_workers=0, pin_memory=torch.cuda.is_available())  

DL_output=pd.DataFrame()
i=0
for sample in val_loader_viz:  
        ids = val_loader_viz.dataset.image_files[i]
        print(ids)
        avg_pool_output = extract_layer_output(model, sample, 'avgpool')  
        avg_pool_output=avg_pool_output.cpu().detach().numpy()
        avg_pool_output=avg_pool_output.reshape(1,-1)
        ids_df = pd.Series(ids)  
        avg_pool_output_df = pd.DataFrame(avg_pool_output)  
        # Concatenate ids and avg_pool_output horizontally  
        combined_df = pd.concat([ids_df.reset_index(drop=True), avg_pool_output_df.reset_index(drop=True)], axis=1)  
        # Append the combined DataFrame to DL_output  
        DL_output = pd.concat([DL_output, combined_df], ignore_index=True) 
        i=i+1

DL_output


,0,0,1,2,3,4,5,6,7,8,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,E:\127lungnode\lungnode\crop3d\patient207_1.ni...,0.906,1.145,0.983,0.956,0.920,0.582,0.986,1.016,1.047,...,1.184,0.708,1.113,0.816,1.135,0.725,1.217,0.697,0.821,1.160
1,E:\127lungnode\lungnode\crop3d\patient207_1.ni...,0.859,1.182,0.966,0.926,0.888,0.599,0.932,0.954,1.015,...,1.166,0.727,1.088,0.797,1.071,0.699,1.180,0.721,0.873,1.171
2,E:\127lungnode\lungnode\crop3d\patient207_1.ni...,0.868,1.153,0.931,0.971,0.916,0.562,0.918,1.104,0.980,...,1.174,0.741,1.126,0.781,1.124,0.774,1.236,0.734,0.791,1.179
3,E:\127lungnode\lungnode\crop3d\patient207_1.ni...,0.856,1.153,0.961,0.944,0.892,0.575,0.970,1.027,1.018,...,1.174,0.700,1.106,0.732,1.114,0.818,1.265,0.712,0.784,1.163
4,E:\127lungnode\lungnode\crop3d\patient207_1.ni...,0.933,1.118,0.955,0.986,0.896,0.568,0.940,0.969,1.053,...,1.125,0.680,1.059,0.767,1.091,0.781,1.160,0.721,0.832,1.156
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,E:\127lungnode\lungnode\crop3d\patient207_1.ni...,0.860,1.175,0.934,0.987,0.917,0.591,0.929,1.046,1.017,...,1.176,0.709,1.082,0.776,1.103,0.770,1.201,0.727,0.794,1.188
93,E:\127lungnode\lungnode\crop3d\patient207_1.ni...,0.877,1.137,0.941,0.951,0.982,0.589,0.905,1.025,1.011,...,1.185,0.711,1.067,0.716,1.089,0.770,1.200,0.698,0.764,1.154
94,E:\127lungnode\lungnode\crop3d\patient207_1.ni...,0.872,1.163,0.977,0.924,0.914,0.581,0.927,1.039,1.013,...,1.181,0.689,1.091,0.760,1.119,0.787,1.244,0.724,0.766,1.154
95,E:\127lungnode\lungnode\crop3d\patient207_1.ni...,0.904,1.122,0.920,0.964,0.928,0.569,0.904,0.984,1.016,...,1.169,0.690,1.075,0.750,1.054,0.764,1.177,0.725,0.822,1.114


In [4]:
DL_output.to_csv('DL_feature.csv',index=False)